In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np


from shapely import wkt

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

### 1. Processing incidents and stations data

In [3]:
incidents = pd.read_csv('Data/WMFS_datasets/wmfs_incidents.csv')
incidents = incidents[incidents['incident_classification_level1'].isin(['FIRE', 'FALSE_ALARM'])]
incidents['call_time'] = pd.to_datetime(incidents['call_time'])

### If only focus on the data in 2018 and 2019, uncomment these lines
# incidents['call_time'] = pd.to_datetime(incidents['call_time'])
# incidents = incidents[(incidents['call_time'].dt.year == 2018) | (incidents['call_time'].dt.year == 2019)]

incidents = incidents.reset_index(drop=True)

In [ ]:
# Define 'total response time'
incidents['total_response_time'] = incidents['reaction_seconds'] + incidents['driving_seconds']
incidents

Keep only the response time and its position, and give them a unique incident_id as the primary key, and convert them into a new geodataframe.

In [4]:
dataset = incidents[['EASTINGS', 'NORTHINGS', 'total_response_time']]
dataset = gpd.GeoDataFrame(dataset, geometry = gpd.points_from_xy(incidents['EASTINGS'], incidents['NORTHINGS']))
dataset = dataset.set_crs('EPSG:27700')
dataset['incident_id'] = range(1, len(dataset) + 1)

dataset = dataset[['incident_id', 'geometry', 'total_response_time']]

In [5]:
dataset

,incident_id,geometry,total_response_time
0,1,POINT (392062.102 286844.969),410
1,2,POINT (405643.149 277939.980),304
2,3,POINT (410260.244 288819.189),221
3,4,POINT (396779.250 299030.106),205
4,5,POINT (410667.961 290492.479),313
...,...,...,...
292692,292693,POINT (398666.687 283225.457),363
292693,292694,POINT (433136.868 277909.031),192
292694,292695,POINT (401134.330 277356.682),379
292695,292696,POINT (406221.059 290654.182),363


In [6]:
stations = pd.read_csv('Data/WMFS_datasets/station_locations.csv')
# stations = stations[stations['Closed (Y/N)'] == 'N']

### 2. Processing OAs and LSOAs

In [8]:
LSOAs = gpd.read_file('Data/LSOAs_boundary/infuse_lsoa_lyr_2011.shp')

,geo_code,geo_label,geo_labelw,label,name,geometry
0,E01003513,Newham 035D,None,E92000001E09000025E01003513,Newham 035D,"POLYGON ((541893.189 181249.621, 541900.568 18..."
1,E01031647,Horsham 002D,None,E92000001E07000227E01031647,Horsham 002D,"POLYGON ((518376.682 132574.695, 518375.785 13..."
2,E01022006,Tendring 002C,None,E92000001E07000076E01022006,Tendring 002C,"POLYGON ((623754.716 231042.037, 623759.750 23..."
3,E01001159,Croydon 002C,None,E92000001E09000008E01001159,Croydon 002C,"POLYGON ((532233.977 170474.976, 532229.824 17..."
4,E01008088,Sheffield 012B,None,E92000001E08000019E01008088,Sheffield 012B,"POLYGON ((433539.233 392096.845, 433539.125 39..."
...,...,...,...,...,...,...
42614,E01029200,South Somerset 004A,None,E92000001E07000189E01029200,South Somerset 004A,"POLYGON ((341784.812 127223.499, 341826.628 12..."
42615,E01003084,Lambeth 030B,None,E92000001E09000022E01003084,Lambeth 030B,"POLYGON ((531528.786 171783.590, 531526.879 17..."
42616,E01020517,West Dorset 007B,None,E92000001E07000052E01020517,West Dorset 007B,"POLYGON ((343718.406 91175.102, 343702.812 911..."
42617,E01004020,Southwark 029D,None,E92000001E09000028E01004020,Southwark 029D,"POLYGON ((534979.720 175304.033, 534980.950 17..."


In [ ]:
LSOAs

Find the LSOAs to which each event occurs and merge it with the processed dataset

In [10]:
dataset_LSOAs = gpd.sjoin(dataset, LSOAs, how = 'left', op='within')
dataset_LSOAs = dataset_LSOAs.drop(columns = 'index_right')
dataset_LSOAs

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,incident_id,geometry,total_response_time,geo_code,geo_label,geo_labelw,label,name,LSOAs_area
0,1,POINT (392062.102 286844.969),410,E01009741,Dudley 022A,None,E92000001E08000027E01009741,Dudley 022A,1.044368e+06
1,2,POINT (405643.149 277939.980),304,E01009110,Birmingham 128B,None,E92000001E08000025E01009110,Birmingham 128B,3.861345e+05
2,3,POINT (410260.244 288819.189),221,E01009479,Birmingham 048B,None,E92000001E08000025E01009479,Birmingham 048B,7.177090e+05
3,4,POINT (396779.250 299030.106),205,E01010408,Walsall 025E,None,E92000001E08000030E01010408,Walsall 025E,3.663721e+05
4,5,POINT (410667.961 290492.479),313,E01008999,Birmingham 031B,None,E92000001E08000025E01008999,Birmingham 031B,3.013927e+05
...,...,...,...,...,...,...,...,...,...
292692,292693,POINT (398666.687 283225.457),363,E01009804,Dudley 036D,None,E92000001E08000027E01009804,Dudley 036D,3.785934e+06
292693,292694,POINT (433136.868 277909.031),192,E01009548,Coventry 031A,None,E92000001E08000026E01009548,Coventry 031A,7.994371e+05
292694,292695,POINT (401134.330 277356.682),379,E01009163,Birmingham 129B,None,E92000001E08000025E01009163,Birmingham 129B,1.542068e+06
292695,292696,POINT (406221.059 290654.182),363,E01009050,Birmingham 035A,None,E92000001E08000025E01009050,Birmingham 035A,3.023399e+05


Calculate the area of ​​LSOAs with fire incidents data and keep only useful columns.

In [ ]:
LSOAs['LSOAs_area'] = LSOAs['geometry'].area

,geo_code,geo_label,geo_labelw,label,name,geometry,LSOAs_area
0,E01003513,Newham 035D,None,E92000001E09000025E01003513,Newham 035D,"POLYGON ((541893.189 181249.621, 541900.568 18...",3.577082e+05
1,E01031647,Horsham 002D,None,E92000001E07000227E01031647,Horsham 002D,"POLYGON ((518376.682 132574.695, 518375.785 13...",1.840764e+05
2,E01022006,Tendring 002C,None,E92000001E07000076E01022006,Tendring 002C,"POLYGON ((623754.716 231042.037, 623759.750 23...",4.368349e+05
3,E01001159,Croydon 002C,None,E92000001E09000008E01001159,Croydon 002C,"POLYGON ((532233.977 170474.976, 532229.824 17...",4.033477e+05
4,E01008088,Sheffield 012B,None,E92000001E08000019E01008088,Sheffield 012B,"POLYGON ((433539.233 392096.845, 433539.125 39...",2.667250e+05
...,...,...,...,...,...,...,...
42614,E01029200,South Somerset 004A,None,E92000001E07000189E01029200,South Somerset 004A,"POLYGON ((341784.812 127223.499, 341826.628 12...",8.367873e+05
42615,E01003084,Lambeth 030B,None,E92000001E09000022E01003084,Lambeth 030B,"POLYGON ((531528.786 171783.590, 531526.879 17...",1.411450e+05
42616,E01020517,West Dorset 007B,None,E92000001E07000052E01020517,West Dorset 007B,"POLYGON ((343718.406 91175.102, 343702.812 911...",2.759288e+07
42617,E01004020,Southwark 029D,None,E92000001E09000028E01004020,Southwark 029D,"POLYGON ((534979.720 175304.033, 534980.950 17...",1.958110e+05


In [ ]:
LSOAs = LSOAs[LSOAs['geo_code'].isin(dataset_LSOAs['geo_code'])].reset_index()
LSOAs = LSOAs[['geo_code', 'geometry', 'LSOAs_area']]

In [ ]:
LSOAs

,geo_code,geometry,LSOAs_area
0,E01009303,"POLYGON ((415398.905 286781.585, 415414.183 28...",494875.672613
1,E01009836,"POLYGON ((392253.243 284253.473, 392247.035 28...",698451.091421
2,E01009183,"POLYGON ((406757.637 283108.192, 406755.064 28...",725677.091024
3,E01008929,"POLYGON ((401289.277 281631.741, 401289.505 28...",532836.286381
4,E01008974,"POLYGON ((406834.124 279656.035, 406839.534 27...",656485.756038
...,...,...,...
1736,E01009548,"POLYGON ((433807.879 278032.854, 433808.969 27...",799437.051189
1737,E01009103,"POLYGON ((414526.813 290912.667, 414534.098 29...",264363.143943
1738,E01009337,"POLYGON ((410261.344 285176.634, 410264.018 28...",152305.244621
1739,E01009903,"POLYGON ((388275.136 284963.116, 388273.500 28...",313120.840399


Match OAs and LSOAs and find out which OAs LSOAs are composed of.

In [13]:
OAs = gpd.read_file('Data/OAs_boundary/infuse_oa_lyr_2011.shp')
OAs = OAs[['geo_code', 'geometry']]

In [14]:
LSOAs_OAs = gpd.sjoin(OAs, LSOAs, how = 'left', op = 'within')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [15]:
LSOAs_OAs

,geo_code_left,geometry,index_right,geo_code_right,LSOAs_area
0,S00091364,"POLYGON ((391280.278 793151.973, 391285.000 79...",NaN,NaN,NaN
1,S00091122,"POLYGON ((381387.031 806367.972, 381106.996 80...",NaN,NaN,NaN
2,S00090435,"POLYGON ((395021.400 814291.900, 395036.800 81...",NaN,NaN,NaN
3,S00088957,"POLYGON ((384833.375 806086.950, 384817.842 80...",NaN,NaN,NaN
4,S00090331,"POLYGON ((383909.418 810708.114, 383963.774 81...",NaN,NaN,NaN
...,...,...,...,...,...
232291,W00003514,"POLYGON ((250226.677 200456.815, 250227.148 20...",NaN,NaN,NaN
232292,W00003847,"POLYGON ((249981.026 212382.225, 249706.682 21...",NaN,NaN,NaN
232293,W00003483,"POLYGON ((242560.429 225418.480, 242543.079 22...",NaN,NaN,NaN
232294,W00003402,"POLYGON ((245283.321 200923.944, 245290.072 20...",NaN,NaN,NaN


In [16]:
LSOAs_OAs = LSOAs_OAs.dropna().reset_index()
LSOAs_OAs = LSOAs_OAs[['geo_code_left', 'geo_code_right']]
LSOAs_OAs = LSOAs_OAs.rename(columns={'geo_code_left': 'OA11CD', 'geo_code_right': 'LSOA11CD'})
LSOAs_OAs

,OA11CD,LSOA11CD
0,E00053291,E01010554
1,E00053210,E01010542
2,E00052886,E01010477
3,E00052807,E01010458
4,E00052579,E01010410
...,...,...
8788,E00052753,E01010448
8789,E00052582,E01010414
8790,E00052627,E01010420
8791,E00053070,E01010506


In [ ]:
output_file_path = 'Data/LSOAs_OAs.csv'
LSOAs_OAs.to_csv(output_file_path, index=False, encoding='utf-8')

### 3. Feature data manipulation

Some features only have data based on LSOAs. In order to maintain the consistency of each feature, we regard the feature of each point as the feature of the LSOAs where it is located.

#### POI

In [17]:
poi = pd.read_csv('Data/poi/poi.csv', delimiter='|', dtype={'pointx_class': str})
poi_classification = pd.read_excel('Data/poi/classification.xlsx', dtype=str)

poi = gpd.GeoDataFrame(poi, geometry=gpd.points_from_xy(poi['feature_easting'], poi['feature_northing']))
poi = poi.set_crs('EPSG:27700')

poi['group'] = poi['pointx_class'].str[:2]

# drop duplicate rows
classification_group = poi_classification[['No_group', 'Group']].drop_duplicates()

# merge group information
poi = poi.merge(classification_group, left_on='group', right_on='No_group', how='left')

poi

,ref_no,name,pointx_class,feature_easting,feature_northing,pos_accuracy,uprn,topo_toid,topo_toid_version,usrn,...,url,brand,qualifier_type,qualifier_data,provenance,supply_date,geometry,group,No_group,Group
0,76327491,Ford,10540734,409945.0,282101.0,2,NaN,Not Assigned,0,2703266,...,NaN,NaN,NaN,NaN,Ordnance Survey,2023-12-01,POINT (409945.000 282101.000),10,10,Transport
1,78437564,Works,07410542,402393.0,288735.0,2,NaN,Not Assigned,0,33608460,...,NaN,NaN,NaN,NaN,Ordnance Survey,2023-12-01,POINT (402393.000 288735.000),07,07,Manufacturing and production
2,78437565,Works,07410542,405887.0,287715.0,2,NaN,Not Assigned,0,2702457,...,NaN,NaN,NaN,NaN,Ordnance Survey,2023-12-01,POINT (405887.000 287715.000),07,07,Manufacturing and production
3,78438083,Works,07410542,403842.0,288198.0,2,NaN,Not Assigned,0,33623420,...,NaN,NaN,NaN,NaN,Ordnance Survey,2023-12-01,POINT (403842.000 288198.000),07,07,Manufacturing and production
4,78438084,Works,07410542,403734.0,288066.0,2,NaN,Not Assigned,0,33600040,...,NaN,NaN,NaN,NaN,Ordnance Survey,2023-12-01,POINT (403734.000 288066.000),07,07,Manufacturing and production
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136342,145881644,Alderman Tye Scout Headquarters,06350452,389075.0,283881.0,1,NaN,osgb1000002075165269,3,11403328,...,www.scouts.org.uk,The Scout Association,NaN,NaN,Ordnance Survey,2023-12-01,POINT (389075.000 283881.000),06,06,Public infrastructure
136343,145881772,Family Community Centre,06340456,415304.6,307077.2,1,NaN,osgb1000025374668,11,23401631,...,NaN,NaN,NaN,NaN,Ordnance Survey,2023-12-01,POINT (415304.600 307077.200),06,06,Public infrastructure
136344,145881663,Neil Dougherty Community Centre,06340456,393393.0,302135.0,1,1.000712e+10,osgb1000024817313,6,44832350,...,NaN,NaN,NaN,NaN,Ordnance Survey,2023-12-01,POINT (393393.000 302135.000),06,06,Public infrastructure
136345,145881918,Great Barr Methodist Church Hall,06340456,404597.0,294411.0,1,3.217615e+07,osgb1000020216141,5,33646700,...,NaN,NaN,NaN,NaN,Ordnance Survey,2023-12-01,POINT (404597.000 294411.000),06,06,Public infrastructure


In [18]:
poi_LSOAs = gpd.sjoin(poi, LSOAs, how = 'right', op='within')
poi_LSOAs = poi_LSOAs.groupby(['geo_code', 'Group']).size().unstack(fill_value=0).reset_index()
poi_LSOAs.columns.name = None
poi_LSOAs

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,geo_code,"Accommodation, eating and drinking",Attractions,Commercial services,Education and health,Manufacturing and production,Public infrastructure,Retail,Sport and entertainment,Transport
0,E01008881,14,2,24,9,1,9,16,1,14
1,E01008882,0,1,2,1,0,4,0,1,5
2,E01008883,5,3,16,7,10,11,6,1,4
3,E01008884,14,2,80,5,57,19,43,2,23
4,E01008885,2,0,18,2,1,2,6,0,3
...,...,...,...,...,...,...,...,...,...,...
1736,E01033646,2,2,14,12,1,5,2,3,10
1737,E01033647,4,0,5,2,0,5,4,1,4
1738,E01033648,13,1,11,4,2,5,10,0,2
1739,E01033649,1,1,4,0,0,4,2,0,0


In [19]:
poi_LSOAs = pd.merge(LSOAs, poi_LSOAs, on='geo_code', how='left')
poi_LSOAs.iloc[:, 3:] = poi_LSOAs.iloc[:, 3:].div(poi_LSOAs['LSOAs_area'], axis=0)
poi_LSOAs = poi_LSOAs.drop(columns=['geometry', 'LSOAs_area'])

poi_LSOAs

,geo_code,"Accommodation, eating and drinking",Attractions,Commercial services,Education and health,Manufacturing and production,Public infrastructure,Retail,Sport and entertainment,Transport
0,E01009303,0.000000,0.000000,0.000036,0.000008,0.000053,0.000016,0.000018,0.000006,0.000020
1,E01009836,0.000047,0.000003,0.000084,0.000017,0.000086,0.000044,0.000059,0.000006,0.000034
2,E01009183,0.000000,0.000008,0.000007,0.000003,0.000000,0.000012,0.000000,0.000007,0.000011
3,E01008929,0.000002,0.000004,0.000015,0.000009,0.000000,0.000011,0.000008,0.000004,0.000032
4,E01008974,0.000005,0.000003,0.000015,0.000005,0.000002,0.000018,0.000005,0.000002,0.000008
...,...,...,...,...,...,...,...,...,...,...
1736,E01009548,0.000010,0.000014,0.000096,0.000020,0.000020,0.000056,0.000013,0.000003,0.000039
1737,E01009103,0.000004,0.000004,0.000023,0.000004,0.000000,0.000019,0.000004,0.000008,0.000023
1738,E01009337,0.000000,0.000000,0.000026,0.000020,0.000000,0.000026,0.000013,0.000000,0.000026
1739,E01009903,0.000010,0.000000,0.000010,0.000003,0.000000,0.000016,0.000003,0.000000,0.000016


#### Land use

In [20]:
land_use = gpd.read_file('Data/land_cover/corine_clip/U2018_CLC2018_V2020_20u1.shp')
land_use = land_use.to_crs('EPSG:27700')
land_use = land_use.drop_duplicates()

In [21]:
corine_label = pd.read_excel('Data/land_cover/corine_clip/corine_label.xlsx')

In [22]:
land_use['Code_18'] = land_use['Code_18'].astype(str)
corine_label['CLC_CODE'] = corine_label['CLC_CODE'].astype(str)

land_use = land_use.merge(corine_label, left_on = 'Code_18', right_on = 'CLC_CODE', how='left')
land_use

,OBJECTID,Code_18,Remark,Area_Ha,ID,Shape_Leng,Shape_Area,geometry,GRID_CODE,CLC_CODE,LABEL1,LABEL2,LABEL3,RGB
0,1569824,111,None,2.262873e+02,EU_1569824,1.039107e+04,2.262873e+06,"POLYGON ((434060.759 281271.159, 434138.964 28...",1,111,Artificial surfaces,Urban fabric,Continuous urban fabric,230-000-077
1,1569827,111,None,5.451204e+01,EU_1569827,3.444423e+03,5.451204e+05,"POLYGON ((436249.950 292550.057, 436483.934 29...",1,111,Artificial surfaces,Urban fabric,Continuous urban fabric,230-000-077
2,1569828,111,None,4.716700e+02,EU_1569828,1.229874e+04,4.716700e+06,"POLYGON ((406812.845 287890.187, 406833.717 28...",1,111,Artificial surfaces,Urban fabric,Continuous urban fabric,230-000-077
3,1569829,111,None,1.286405e+02,EU_1569829,7.865725e+03,1.286405e+06,"POLYGON ((394968.649 290799.161, 395051.262 29...",1,111,Artificial surfaces,Urban fabric,Continuous urban fabric,230-000-077
4,1569830,111,None,1.158835e+02,EU_1569830,6.061519e+03,1.158835e+06,"POLYGON ((399683.262 292156.593, 399819.933 29...",1,111,Artificial surfaces,Urban fabric,Continuous urban fabric,230-000-077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,1625571,512,None,1.004947e+02,EU_1625571,6.665282e+03,1.004947e+06,"POLYGON ((404039.530 307726.122, 404082.436 30...",41,512,Water bodies,Inland waters,Water bodies,128-242-230
1047,1625572,512,None,2.960400e+01,EU_1625572,2.845593e+03,2.960400e+05,"POLYGON ((385386.854 306238.103, 385442.841 30...",41,512,Water bodies,Inland waters,Water bodies,128-242-230
1048,1625582,512,None,3.922768e+01,EU_1625582,2.924718e+03,3.922768e+05,"POLYGON ((393824.077 310043.220, 393746.260 31...",41,512,Water bodies,Inland waters,Water bodies,128-242-230
1049,1625584,512,None,6.385081e+01,EU_1625584,4.189651e+03,6.385081e+05,"POLYGON ((387009.414 310053.251, 387023.865 31...",41,512,Water bodies,Inland waters,Water bodies,128-242-230


In [23]:
land_use_LSOAs = gpd.overlay(LSOAs, land_use, how='intersection')
land_use_LSOAs['area'] = land_use_LSOAs['geometry'].area
land_use_LSOAs['land_use_percentage'] = (land_use_LSOAs['area'] / land_use_LSOAs['LSOAs_area']) * 100

land_use_LSOAs = land_use_LSOAs.groupby(['geo_code', 'LABEL1'])['land_use_percentage'].sum()
land_use_LSOAs = land_use_LSOAs.unstack(fill_value=0).reset_index()
land_use_LSOAs.columns.name = None

land_use_LSOAs

,geo_code,Agricultural areas,Artificial surfaces,Forest and semi natural areas,Water bodies
0,E01008881,0.0,100.0,0.0,0.0
1,E01008882,0.0,100.0,0.0,0.0
2,E01008883,0.0,100.0,0.0,0.0
3,E01008884,0.0,100.0,0.0,0.0
4,E01008885,0.0,100.0,0.0,0.0
...,...,...,...,...,...
1736,E01033646,0.0,100.0,0.0,0.0
1737,E01033647,0.0,100.0,0.0,0.0
1738,E01033648,0.0,100.0,0.0,0.0
1739,E01033649,0.0,100.0,0.0,0.0


#### Buildings

In [24]:
buildings = gpd.read_file('Data/buildings/buildings.gpkg')

In [25]:
### Calculate the centroid of buildings
buildings['centroid'] = buildings.geometry.centroid
buildings = buildings.drop(columns = 'geometry')
buildings = buildings.set_geometry('centroid')

# find buildings are in which grid
buildings_LSOAs= gpd.sjoin(buildings, LSOAs, how='right', op='within')
buildings_LSOAs['count'] = buildings_LSOAs['unique_property_number'].notna().astype(int)

# count the buildings
buildings_LSOAs = buildings_LSOAs.groupby('geo_code')['count'].sum().reset_index(name = 'building_count')
buildings_LSOAs

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,geo_code,building_count
0,E01008881,641
1,E01008882,739
2,E01008883,734
3,E01008884,909
4,E01008885,805
...,...,...
1736,E01033646,428
1737,E01033647,490
1738,E01033648,717
1739,E01033649,536


In [26]:
buildings_LSOAs = pd.merge(LSOAs, buildings_LSOAs, on='geo_code', how='left')
buildings_LSOAs.iloc[:, 3:] = buildings_LSOAs.iloc[:, 3:].div(buildings_LSOAs['LSOAs_area'], axis=0)
buildings_LSOAs = buildings_LSOAs.drop(columns=['geometry', 'LSOAs_area'])

buildings_LSOAs

,geo_code,building_count
0,E01009303,0.001574
1,E01009836,0.000793
2,E01009183,0.000898
3,E01008929,0.001096
4,E01008974,0.001599
...,...,...
1736,E01009548,0.001132
1737,E01009103,0.002398
1738,E01009337,0.003493
1739,E01009903,0.002398


#### Total rented

In [27]:
rent_LSOAs = pd.read_excel('Data/rent/rent.xlsx', sheet_name = '1c', skiprows = 2)

In [28]:
rent_LSOAs = rent_LSOAs.replace('c', 5)
rent_LSOAs['total_rent'] = rent_LSOAs['Social rented'] + rent_LSOAs['Private rented or lives rent free']
rent_LSOAs = rent_LSOAs[rent_LSOAs['LSOA code'].isin(LSOAs['geo_code'])]
rent_LSOAs

,LSOA code,LSOA name,Owned: Owns outright,"Owned: Owns with a mortgage, loan or shared ownership",Social rented,Private rented or lives rent free,total_rent
8449,E01008881,Birmingham 067A,160,165,135,125,260
8450,E01008882,Birmingham 066A,125,180,225,145,370
8451,E01008883,Birmingham 078A,145,225,110,160,270
8452,E01008884,Birmingham 078B,100,145,165,300,465
8453,E01008885,Birmingham 076A,205,195,5,95,100
...,...,...,...,...,...,...,...
31711,E01033643,Birmingham 082F,260,150,160,250,410
31712,E01033646,Birmingham 031I,90,95,295,375,670
31713,E01033648,Birmingham 084F,260,90,315,165,480
31714,E01033649,Birmingham 058F,120,140,80,155,235


In [29]:
rent_LSOAs = pd.merge(LSOAs, rent_LSOAs, left_on='geo_code', right_on='LSOA code', how='left').fillna(0)
rent_LSOAs = rent_LSOAs[['geo_code', 'total_rent']]
rent_LSOAs

,geo_code,total_rent
0,E01009303,345.0
1,E01009836,675.0
2,E01009183,130.0
3,E01008929,460.0
4,E01008974,325.0
...,...,...
1736,E01009548,500.0
1737,E01009103,515.0
1738,E01009337,260.0
1739,E01009903,50.0


In [30]:
rent_LSOAs = pd.merge(LSOAs, rent_LSOAs, on='geo_code', how='left')
rent_LSOAs.iloc[:, 3:] = rent_LSOAs.iloc[:, 3:].div(rent_LSOAs['LSOAs_area'], axis=0)
rent_LSOAs = rent_LSOAs.drop(columns=['geometry', 'LSOAs_area'])

rent_LSOAs

,geo_code,total_rent
0,E01009303,0.000697
1,E01009836,0.000966
2,E01009183,0.000179
3,E01008929,0.000863
4,E01008974,0.000495
...,...,...
1736,E01009548,0.000625
1737,E01009103,0.001948
1738,E01009337,0.001707
1739,E01009903,0.000160


#### Population (children below 14 and elderly above 65)

In [31]:
# Load 2020 Census data based on OAs
population = pd.read_excel('Data/population/census_OA_2020.xlsx', sheet_name = 'Mid-2020 Persons', skiprows = 4)

In [32]:
pop_LSOAs = population
pop_LSOAs['below_14'] = pop_LSOAs.loc[:, 0:14].sum(axis=1)
pop_LSOAs['above_65'] = pop_LSOAs.loc[:, 65:'90+'].sum(axis=1)
pop_LSOAs = pop_LSOAs[pop_LSOAs['LSOA11CD'].isin(LSOAs['geo_code'])]
pop_LSOAs = pop_LSOAs[['LSOA11CD', 'below_14', 'above_65']]

pop_LSOAs

,LSOA11CD,below_14,above_65
0,E01008881,64,60
1,E01008881,82,37
2,E01008881,143,39
3,E01008881,76,61
4,E01008881,59,46
...,...,...,...
17911,E01033650,49,20
17912,E01033650,100,18
17913,E01033650,116,43
17914,E01033650,113,34


In [33]:
pop_LSOAs = pop_LSOAs.groupby('LSOA11CD').agg(
    below_14 = ('below_14', 'sum'),
    above_65 = ('above_65', 'sum')
    ).reset_index()

pop_LSOAs = pop_LSOAs.rename(columns={'LSOA11CD': 'geo_code'})

pop_LSOAs

,geo_code,below_14,above_65
0,E01008881,424,243
1,E01008882,534,167
2,E01008883,550,206
3,E01008884,694,132
4,E01008885,282,221
...,...,...,...
1736,E01033646,394,200
1737,E01033647,526,102
1738,E01033648,665,244
1739,E01033649,660,76


In [34]:
pop_LSOAs = pd.merge(LSOAs, pop_LSOAs, on='geo_code', how='left')
pop_LSOAs.iloc[:, 3:] = pop_LSOAs.iloc[:, 3:].div(pop_LSOAs['LSOAs_area'], axis=0)
pop_LSOAs = pop_LSOAs.drop(columns=['geometry', 'LSOAs_area'])

pop_LSOAs

,geo_code,below_14,above_65
0,E01009303,0.000639,0.000463
1,E01009836,0.000747,0.000381
2,E01009183,0.000390,0.000347
3,E01008929,0.000642,0.000430
4,E01008974,0.000850,0.000516
...,...,...,...
1736,E01009548,0.000454,0.000306
1737,E01009103,0.001263,0.000889
1738,E01009337,0.003178,0.000827
1739,E01009903,0.000562,0.001460


#### Index of multiple deprivation score

In [35]:
IMD_LSOAs = pd.read_excel('Data/IMD/IoD_2019.xlsx', sheet_name = 'IoD2019 Scores')

In [36]:
IMD_LSOAs = IMD_LSOAs[IMD_LSOAs['LSOA code (2011)'].isin(LSOAs['geo_code'])]
IMD_LSOAs = IMD_LSOAs[['LSOA code (2011)', 'Index of Multiple Deprivation (IMD) Score']]
IMD_LSOAs = IMD_LSOAs.rename(columns={'LSOA code (2011)': 'geo_code'})

IMD_LSOAs

,geo_code,Index of Multiple Deprivation (IMD) Score
8648,E01008881,41.179
8649,E01008882,59.693
8650,E01008883,38.636
8651,E01008884,44.315
8652,E01008885,22.921
...,...,...
32721,E01033646,64.138
32722,E01033647,53.340
32723,E01033648,55.778
32724,E01033649,52.028


#### Street network density

In [37]:
road_link = pd.read_csv('Data/road/road_link.csv')

/var/folders/rv/j_vzprd56j5fpq5_0hvsy8v80000gn/T/ipykernel_16596/1407166712.py:1: DtypeWarning: Columns (5,6,7,15) have mixed types. Specify dtype option on import or set low_memory=False.
  road_link = pd.read_csv('Data/road/road_link.csv')


In [38]:
road_link['geometry'] = road_link['geometry'].apply(wkt.loads)
road_link = gpd.GeoDataFrame(road_link, geometry='geometry')

In [39]:
road_node = pd.read_csv('Data/road/road_node.csv')

In [40]:
road_node['geometry'] = road_node['geometry'].apply(wkt.loads)
road_node = gpd.GeoDataFrame(road_node, geometry='geometry')

In [41]:
road_node_filterd = road_node[road_node['formOfNode'].isin(['junction', 'road end', 'roundabout'])]

streetnet_density = gpd.sjoin(road_node_filterd, LSOAs, how = 'right', op='within')
streetnet_density['count'] = streetnet_density['identifier'].notna().astype(int)

streetnet_density = streetnet_density.groupby('geo_code')['count'].sum().reset_index(name='node_count')
streetnet_density_LSOAs = pd.merge(LSOAs, streetnet_density, on='geo_code', how='left')
streetnet_density_LSOAs['street_network_density'] = streetnet_density_LSOAs['node_count']/streetnet_density_LSOAs['LSOAs_area']
streetnet_density_LSOAs = streetnet_density_LSOAs[['geo_code', 'street_network_density']]

streetnet_density_LSOAs

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/rv/j_vzprd56j5fpq5_0hvsy8v80000gn/T/ipykernel_16596/3966105962.py:3: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:27700

  streetnet_density = gpd.sjoin(road_node_filterd, LSOAs, how = 'right', op='within')


,geo_code,street_network_density
0,E01009303,0.000129
1,E01009836,0.000109
2,E01009183,0.000050
3,E01008929,0.000086
4,E01008974,0.000116
...,...,...
1736,E01009548,0.000198
1737,E01009103,0.000223
1738,E01009337,0.000020
1739,E01009903,0.000128


#### Street connectivity

In [42]:
road_link_LSOAs = gpd.GeoDataFrame(columns=road_link.columns)

# clip the road by each grid
for i, LSOAs_row in LSOAs.iterrows():
    clipped = gpd.clip(road_link, LSOAs_row.geometry)
    clipped['geo_code'] = LSOAs_row['geo_code']
    road_link_LSOAs = pd.concat([road_link_LSOAs, clipped])

road_link_LSOAs = road_link_LSOAs.reset_index()
road_link_LSOAs

,index,fictitious,identifier,class,roadNumber,name1,name1_lang,name2,name2_lang,formOfWay,...,trunkRoad,loop,startNode,endNode,structure,nameTOID,numberTOID,function,geometry,geo_code
0,768491,False,98CBDE5F-A2A9-4632-91F9-A861F20AD0E2,Unclassified,NaN,Mackadown Lane,NaN,NaN,NaN,Single Carriageway,...,False,False,22EA7E24-A3F7-4223-8123-A34AD77C2E9F,31ABD71A-3684-4AB4-A208-7030BEE82F3E,NaN,osgb4000000019393874,NaN,Minor Road,"LINESTRING Z (415879.271 286138.296 0.000, 415...",E01009303
1,768332,False,2B936AB5-E988-459D-9100-55404E0FA85B,Unclassified,NaN,Fastmoor Oval,NaN,NaN,NaN,Single Carriageway,...,False,False,7E03DC75-8AF5-4863-BCCD-62CF7A836F43,BC083AFD-9463-4D33-8FEB-B478CF8277C4,NaN,osgb4000000019414790,NaN,Local Road,"LINESTRING Z (416118.140 286127.780 0.000, 416...",E01009303
2,768508,False,EC13DD46-3E4C-4D99-889B-4F17E31AE5E3,Unknown,NaN,Tile Cross Road,NaN,NaN,NaN,Single Carriageway,...,False,False,494E0335-4B50-4BB8-86DD-80010CDC5CD6,79F273C7-9E7F-42AD-AA82-FC4282ECBB8D,NaN,osgb4000000019396652,NaN,Local Road,"LINESTRING Z (416049.000 286136.000 0.000, 416...",E01009303
3,768335,False,0FCE6AD1-9A89-443D-9412-936BE4BB6A0E,Unclassified,NaN,Fastmoor Oval,NaN,NaN,NaN,Single Carriageway,...,False,False,E1BD7AE7-5F7F-4A5D-B7AE-B957E6C0DAC6,7E03DC75-8AF5-4863-BCCD-62CF7A836F43,NaN,osgb4000000019414790,NaN,Local Road,"LINESTRING Z (416118.190 286127.809 0.000, 416...",E01009303
4,768327,False,D0F2ADF0-3ED2-4C77-A1CE-EC0CB697F3FA,Unknown,NaN,Tile Cross Road,NaN,NaN,NaN,Single Carriageway,...,False,False,1B4C9686-84A2-4FAC-96E9-69FF56541EAE,494E0335-4B50-4BB8-86DD-80010CDC5CD6,NaN,osgb4000000019396652,NaN,Local Road,"LINESTRING Z (416083.140 286156.490 0.000, 416...",E01009303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148951,767104,False,7E68FF88-C7D9-422F-9B3A-E8E1D836D13B,Unclassified,NaN,Brays Road,NaN,NaN,NaN,Roundabout,...,False,False,FFF64BC5-AC2F-4EFC-8683-BBBEA7B1F5E0,E1D8AFB5-0C03-4E72-9116-7914BEF91C3E,NaN,osgb4000000019393868,NaN,Minor Road,MULTILINESTRING Z ((414780.000 284892.000 0.00...,E01009316
148952,767108,False,E3033F48-C843-4672-A45D-F1324A146EC3,Unknown,NaN,NaN,NaN,NaN,NaN,Single Carriageway,...,False,False,1EC4F0C1-2DF6-434F-8F0D-5976B9C70FE2,5B546C5D-8B9E-403B-9DA0-06BDDC2285F4,NaN,NaN,NaN,Secondary Access Road,"LINESTRING Z (414574.046 284908.652 0.000, 414...",E01009316
148953,767105,False,74783F2E-C7B8-462C-9D1C-7F7C0FD3E409,Unclassified,NaN,Brays Road,NaN,NaN,NaN,Single Carriageway,...,False,False,FFF64BC5-AC2F-4EFC-8683-BBBEA7B1F5E0,63469137-1E18-4449-8FEA-9BB5BB434F39,NaN,osgb4000000019393868,NaN,Minor Road,POINT Z (414780.000 284892.000 0.000),E01009316
148954,767106,False,4E381F2E-7742-4A53-9ADB-573C5FF0F3E5,Unclassified,NaN,Benedon Road,NaN,NaN,NaN,Single Carriageway,...,False,False,345AF225-95BA-45B2-9DC4-187ED6B6DB05,3581B232-8902-476F-9970-5F3110411FB5,NaN,osgb4000000019396619,NaN,Local Road,"LINESTRING Z (414691.033 284943.635 0.000, 414...",E01009316


In [43]:
road_link_count = road_link_LSOAs.groupby('geo_code').size().reset_index(name='link_count')
road_link_count = pd.merge(LSOAs, road_link_count, on='geo_code', how='left').fillna(0)

road_link_count

,geo_code,geometry,LSOAs_area,link_count
0,E01009303,"POLYGON ((415398.905 286781.585, 415414.183 28...",494875.672613,91
1,E01009836,"POLYGON ((392253.243 284253.473, 392247.035 28...",698451.091421,126
2,E01009183,"POLYGON ((406757.637 283108.192, 406755.064 28...",725677.091024,68
3,E01008929,"POLYGON ((401289.277 281631.741, 401289.505 28...",532836.286381,93
4,E01008974,"POLYGON ((406834.124 279656.035, 406839.534 27...",656485.756038,125
...,...,...,...,...
1736,E01009548,"POLYGON ((433807.879 278032.854, 433808.969 27...",799437.051189,257
1737,E01009103,"POLYGON ((414526.813 290912.667, 414534.098 29...",264363.143943,90
1738,E01009337,"POLYGON ((410261.344 285176.634, 410264.018 28...",152305.244621,18
1739,E01009903,"POLYGON ((388275.136 284963.116, 388273.500 28...",313120.840399,61


In [44]:
street_con_LSOAs = pd.merge(streetnet_density, road_link_count, on = 'geo_code', how = 'inner')
street_con_LSOAs['connectivity'] = street_con_LSOAs['link_count'] / street_con_LSOAs['node_count']

street_con_LSOAs = street_con_LSOAs[['geo_code', 'connectivity']].fillna(0)

street_con_LSOAs

,geo_code,connectivity
0,E01008881,1.590164
1,E01008882,1.515152
2,E01008883,1.724138
3,E01008884,1.634615
4,E01008885,1.520000
...,...,...
1736,E01033646,2.071429
1737,E01033647,2.384615
1738,E01033648,2.066667
1739,E01033649,2.833333


#### Road density

In [49]:
road_link_LSOAs['road_length'] = road_link_LSOAs['geometry'].length
road_density_LSOAs = road_link_LSOAs.groupby('geo_code')['road_length'].sum().reset_index()

road_density_LSOAs = pd.merge(LSOAs, road_density_LSOAs, on='geo_code', how='left').fillna(0)
road_density_LSOAs['road_density'] = road_density_LSOAs['road_length']/road_density_LSOAs['LSOAs_area']
road_density_LSOAs = road_density_LSOAs[['geo_code', 'road_density']]

road_density_LSOAs

,geo_code,road_density
0,E01009303,0.011738
1,E01009836,0.011281
2,E01009183,0.010348
3,E01008929,0.010647
4,E01008974,0.012835
...,...,...
1736,E01009548,0.021072
1737,E01009103,0.017775
1738,E01009337,0.010916
1739,E01009903,0.014921


#### Nearest fire station distance

In [50]:
station_gdf = gpd.GeoDataFrame(
    stations, 
    geometry=gpd.points_from_xy(stations['Easting'], stations['Northing']),
    crs='EPSG:27700')

In [51]:
### Define a function to calculation the distance of the incident to the nearest fire station
def nearest_distance(incident, stations_gdf):
       
    distances = stations_gdf.geometry.distance(incident)
    return distances.min()

distance = dataset_LSOAs.copy()
distance['nearest_station_distance'] = distance['geometry'].apply(nearest_distance, stations_gdf = station_gdf)


In [52]:
distance = distance[['incident_id', 'nearest_station_distance']].reset_index(drop = True)
distance

,incident_id,nearest_station_distance
0,1,951.314076
1,2,1915.224736
2,3,1070.416730
3,4,699.625584
4,5,1673.295243
...,...,...
292692,292693,2928.773956
292693,292694,1547.944724
292694,292695,1622.377785
292695,292696,1445.160192


#### Frequency of fire incidents in the neighborhood (750m radius)

In [53]:
distance_neighbor = 750

buffer_neighbor = dataset_LSOAs.copy()

buffer_neighbor['buffer'] = buffer_neighbor['geometry'].buffer(distance_neighbor)
buffer_neighbor = gpd.GeoDataFrame(buffer_neighbor, geometry='buffer', crs = dataset_LSOAs.crs)
buffer_neighbor

,incident_id,geometry,total_response_time,geo_code,geo_label,geo_labelw,label,name,LSOAs_area,buffer
0,1,POINT (392062.102 286844.969),410,E01009741,Dudley 022A,None,E92000001E08000027E01009741,Dudley 022A,1.044368e+06,"POLYGON ((392812.102 286844.969, 392808.491 28..."
1,2,POINT (405643.149 277939.980),304,E01009110,Birmingham 128B,None,E92000001E08000025E01009110,Birmingham 128B,3.861345e+05,"POLYGON ((406393.149 277939.980, 406389.538 27..."
2,3,POINT (410260.244 288819.189),221,E01009479,Birmingham 048B,None,E92000001E08000025E01009479,Birmingham 048B,7.177090e+05,"POLYGON ((411010.244 288819.189, 411006.633 28..."
3,4,POINT (396779.250 299030.106),205,E01010408,Walsall 025E,None,E92000001E08000030E01010408,Walsall 025E,3.663721e+05,"POLYGON ((397529.250 299030.106, 397525.639 29..."
4,5,POINT (410667.961 290492.479),313,E01008999,Birmingham 031B,None,E92000001E08000025E01008999,Birmingham 031B,3.013927e+05,"POLYGON ((411417.961 290492.479, 411414.350 29..."
...,...,...,...,...,...,...,...,...,...,...
292692,292693,POINT (398666.687 283225.457),363,E01009804,Dudley 036D,None,E92000001E08000027E01009804,Dudley 036D,3.785934e+06,"POLYGON ((399416.687 283225.457, 399413.075 28..."
292693,292694,POINT (433136.868 277909.031),192,E01009548,Coventry 031A,None,E92000001E08000026E01009548,Coventry 031A,7.994371e+05,"POLYGON ((433886.868 277909.031, 433883.256 27..."
292694,292695,POINT (401134.330 277356.682),379,E01009163,Birmingham 129B,None,E92000001E08000025E01009163,Birmingham 129B,1.542068e+06,"POLYGON ((401884.330 277356.682, 401880.719 27..."
292695,292696,POINT (406221.059 290654.182),363,E01009050,Birmingham 035A,None,E92000001E08000025E01009050,Birmingham 035A,3.023399e+05,"POLYGON ((406971.059 290654.182, 406967.447 29..."


In [54]:
from shapely.strtree import STRtree
spatial_index = STRtree(buffer_neighbor.geometry)

def count_points_within_buffer(geom, spatial_index):
    points_within_buffer = spatial_index.query(geom)
    return len(points_within_buffer) - 1

buffer_neighbor['frequency'] = buffer_neighbor['geometry'].apply(lambda geom: count_points_within_buffer(geom, spatial_index))

In [55]:
fire_neighbor = buffer_neighbor[['incident_id', 'frequency']]
fire_neighbor['frequency'] = fire_neighbor['frequency'].apply(lambda x: x/(15*12))
fire_neighbor

/var/folders/rv/j_vzprd56j5fpq5_0hvsy8v80000gn/T/ipykernel_16596/2212698120.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fire_neighbor['frequency'] = fire_neighbor['frequency'].apply(lambda x: x/(15*12))


,incident_id,frequency
0,1,7.661111
1,2,3.555556
2,3,8.105556
3,4,8.483333
4,5,7.666667
...,...,...
292692,292693,1.622222
292693,292694,6.216667
292694,292695,3.522222
292695,292696,7.777778


#### The number of the fire stations near the incident

In [58]:
distance_station = 10000

buffer_station = dataset_LSOAs.copy()

buffer_station['buffer'] = buffer_station['geometry'].buffer(distance_station)
buffer_station = gpd.GeoDataFrame(buffer_station, geometry='buffer', crs = dataset_LSOAs.crs)
buffer_station

,incident_id,geometry,total_response_time,geo_code,geo_label,geo_labelw,label,name,LSOAs_area,buffer
0,1,POINT (392062.102 286844.969),410,E01009741,Dudley 022A,None,E92000001E08000027E01009741,Dudley 022A,1.044368e+06,"POLYGON ((402062.102 286844.969, 402013.949 28..."
1,2,POINT (405643.149 277939.980),304,E01009110,Birmingham 128B,None,E92000001E08000025E01009110,Birmingham 128B,3.861345e+05,"POLYGON ((415643.149 277939.980, 415594.997 27..."
2,3,POINT (410260.244 288819.189),221,E01009479,Birmingham 048B,None,E92000001E08000025E01009479,Birmingham 048B,7.177090e+05,"POLYGON ((420260.244 288819.189, 420212.092 28..."
3,4,POINT (396779.250 299030.106),205,E01010408,Walsall 025E,None,E92000001E08000030E01010408,Walsall 025E,3.663721e+05,"POLYGON ((406779.250 299030.106, 406731.098 29..."
4,5,POINT (410667.961 290492.479),313,E01008999,Birmingham 031B,None,E92000001E08000025E01008999,Birmingham 031B,3.013927e+05,"POLYGON ((420667.961 290492.479, 420619.809 28..."
...,...,...,...,...,...,...,...,...,...,...
292692,292693,POINT (398666.687 283225.457),363,E01009804,Dudley 036D,None,E92000001E08000027E01009804,Dudley 036D,3.785934e+06,"POLYGON ((408666.687 283225.457, 408618.534 28..."
292693,292694,POINT (433136.868 277909.031),192,E01009548,Coventry 031A,None,E92000001E08000026E01009548,Coventry 031A,7.994371e+05,"POLYGON ((443136.868 277909.031, 443088.715 27..."
292694,292695,POINT (401134.330 277356.682),379,E01009163,Birmingham 129B,None,E92000001E08000025E01009163,Birmingham 129B,1.542068e+06,"POLYGON ((411134.330 277356.682, 411086.178 27..."
292695,292696,POINT (406221.059 290654.182),363,E01009050,Birmingham 035A,None,E92000001E08000025E01009050,Birmingham 035A,3.023399e+05,"POLYGON ((416221.059 290654.182, 416172.906 28..."


In [59]:
station_num = gpd.sjoin(station_gdf, buffer_station, how = 'right', op = 'within')
station_num = station_num.groupby('incident_id').size().reset_index(name = 'station_count')

station_num

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,incident_id,station_count
0,1,11
1,2,9
2,3,14
3,4,12
4,5,13
...,...,...
292692,292693,16
292693,292694,4
292694,292695,7
292695,292696,19


#### Aggregate all features

In [74]:
features = [poi_LSOAs, buildings_LSOAs, land_use_LSOAs, rent_LSOAs, pop_LSOAs, IMD_LSOAs, streetnet_density_LSOAs,
            street_con_LSOAs, road_density_LSOAs]

# start with the incident DataFrame
features_all = dataset_LSOAs[['incident_id', 'geo_code', 'total_response_time']]

# merge each subsequent DataFrame
for df in features:
    features_all = pd.merge(features_all, df, on='geo_code', how='left')

features_all

,incident_id,geo_code,total_response_time,"Accommodation, eating and drinking",Attractions,Commercial services,Education and health,Manufacturing and production,Public infrastructure,Retail,...,Artificial surfaces,Forest and semi natural areas,Water bodies,total_rent,below_14,above_65,Index of Multiple Deprivation (IMD) Score,street_network_density,connectivity,road_density
0,1,E01009741,410,6.798372e-05,0.000005,0.000103,1.915034e-05,0.000031,0.000053,0.000165,...,100.000000,0.000000,0.0,0.000804,0.000411,0.000251,43.356,0.000119,1.556452,0.013113
1,2,E01009110,304,0.000000e+00,0.000000,0.000008,1.035908e-05,0.000000,0.000013,0.000003,...,60.278864,0.000000,0.0,0.000777,0.000650,0.000578,50.571,0.000153,1.525424,0.015970
2,3,E01009479,221,6.966611e-06,0.000001,0.000008,2.786645e-06,0.000006,0.000010,0.000014,...,78.388729,0.000000,0.0,0.000453,0.000874,0.000223,54.069,0.000045,1.875000,0.008800
3,4,E01010408,205,8.188396e-06,0.000000,0.000082,0.000000e+00,0.000066,0.000027,0.000016,...,100.000000,0.000000,0.0,0.001078,0.001015,0.000478,46.226,0.000134,2.061224,0.014677
4,5,E01008999,313,0.000000e+00,0.000000,0.000020,2.322551e-05,0.000000,0.000010,0.000007,...,100.000000,0.000000,0.0,0.000581,0.001082,0.000610,42.242,0.000116,2.057143,0.013002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292692,292693,E01009804,363,7.924069e-07,0.000002,0.000005,7.924069e-07,0.000002,0.000005,0.000002,...,21.038698,0.007215,0.0,0.000013,0.000036,0.000100,7.874,0.000020,1.693333,0.003569
292693,292694,E01009548,192,1.000704e-05,0.000014,0.000096,2.001408e-05,0.000020,0.000056,0.000013,...,100.000000,0.000000,0.0,0.000625,0.000454,0.000306,16.260,0.000198,1.626582,0.021072
292694,292695,E01009163,379,8.430237e-06,0.000006,0.000016,3.242399e-06,0.000002,0.000014,0.000012,...,65.093673,0.823200,0.0,0.000000,0.000305,0.000336,43.091,0.000069,1.745283,0.008614
292695,292696,E01009050,363,3.307536e-06,0.000003,0.000010,3.307536e-06,0.000007,0.000056,0.000007,...,100.000000,0.000000,0.0,0.001538,0.001654,0.000549,42.623,0.000103,1.967742,0.012185


In [75]:
features_all = pd.merge(features_all, distance, on='incident_id', how='left')
features_all = pd.merge(features_all, fire_neighbor, on='incident_id', how='left')
features_all = pd.merge(features_all, station_num, on='incident_id', how='left')

features_all = features_all.drop(columns = ['incident_id', 'geo_code'])

features_all

,total_response_time,"Accommodation, eating and drinking",Attractions,Commercial services,Education and health,Manufacturing and production,Public infrastructure,Retail,Sport and entertainment,Transport,...,total_rent,below_14,above_65,Index of Multiple Deprivation (IMD) Score,street_network_density,connectivity,road_density,nearest_station_distance,frequency,station_count
0,410,6.798372e-05,0.000005,0.000103,1.915034e-05,0.000031,0.000053,0.000165,0.000011,0.000052,...,0.000804,0.000411,0.000251,43.356,0.000119,1.556452,0.013113,951.314076,7.661111,11
1,304,0.000000e+00,0.000000,0.000008,1.035908e-05,0.000000,0.000013,0.000003,0.000000,0.000013,...,0.000777,0.000650,0.000578,50.571,0.000153,1.525424,0.015970,1915.224736,3.555556,9
2,221,6.966611e-06,0.000001,0.000008,2.786645e-06,0.000006,0.000010,0.000014,0.000000,0.000010,...,0.000453,0.000874,0.000223,54.069,0.000045,1.875000,0.008800,1070.416730,8.105556,14
3,205,8.188396e-06,0.000000,0.000082,0.000000e+00,0.000066,0.000027,0.000016,0.000005,0.000022,...,0.001078,0.001015,0.000478,46.226,0.000134,2.061224,0.014677,699.625584,8.483333,12
4,313,0.000000e+00,0.000000,0.000020,2.322551e-05,0.000000,0.000010,0.000007,0.000007,0.000020,...,0.000581,0.001082,0.000610,42.242,0.000116,2.057143,0.013002,1673.295243,7.666667,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292692,363,7.924069e-07,0.000002,0.000005,7.924069e-07,0.000002,0.000005,0.000002,0.000003,0.000009,...,0.000013,0.000036,0.000100,7.874,0.000020,1.693333,0.003569,2928.773956,1.622222,16
292693,192,1.000704e-05,0.000014,0.000096,2.001408e-05,0.000020,0.000056,0.000013,0.000003,0.000039,...,0.000625,0.000454,0.000306,16.260,0.000198,1.626582,0.021072,1547.944724,6.216667,4
292694,379,8.430237e-06,0.000006,0.000016,3.242399e-06,0.000002,0.000014,0.000012,0.000003,0.000020,...,0.000000,0.000305,0.000336,43.091,0.000069,1.745283,0.008614,1622.377785,3.522222,7
292695,363,3.307536e-06,0.000003,0.000010,3.307536e-06,0.000007,0.000056,0.000007,0.000003,0.000010,...,0.001538,0.001654,0.000549,42.623,0.000103,1.967742,0.012185,1445.160192,7.777778,19


### 4. Modelling

In [76]:
### Separate dependent and independt variables
X = features_all.drop(columns= 'total_response_time')
y = features_all['total_response_time']

#### Lasso regression

Lasso regression itself has the  ability of selecting important features, so it can be used directly. First, we need to initialize the intercept and add a constant to each set of data.

In [ ]:
X_regression = sm.add_constant(X)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_regression, y, test_size=0.2, random_state=0)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the Lasso model with cross-validation to find the best lambda (alpha)
lasso = LassoCV(cv=5, random_state=0)
lasso.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = lasso.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'R²: {r2}')
print(f'Best lambda (alpha): {lasso.alpha_}')

MSE: 3534.2821383268624
R²: 0.7663045571963532
Best lambda (alpha): 0.10372305242028328


#### Tree-based model

##### I. Feature selection


First, use random forest to select important features.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# select the features using random forest
rf = RandomForestRegressor(n_estimators=100, random_state=0)
rf.fit(X_train, y_train)

selector = SelectFromModel(rf, threshold='median')
selector.fit(X_train, y_train)

X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

In [ ]:
# the value of feature importance
feature_importances = rf.feature_importances_
selected_features_mask = selector.get_support()

# get the feature importance
selected_feature_importances = feature_importances[selected_features_mask]

selected_feature_indices = np.where(selected_features_mask)[0]
selected_feature_names = np.array(X.columns)[selected_feature_indices]

sorted_indices = np.argsort(selected_feature_importances)[::-1]
sorted_importances = selected_feature_importances[sorted_indices]
sorted_feature_names = selected_feature_names[sorted_indices]

# print the selected features and their importance
for name, importance in zip(sorted_feature_names, sorted_importances):
    print(f"Name: {name}, Importance: {importance}")

Name: nearest_station_distance, Importance: 0.6456623754955758
Name: neighbour_frequency_per_month, Importance: 0.17321816200192305
Name: below_14, Importance: 0.021277039134290326
Name: road_length, Importance: 0.0196974926061004
Name: above_65, Importance: 0.01463205609845586
Name: total_rent, Importance: 0.014600111042507297
Name: IMD, Importance: 0.013560881429045086
Name: building_count, Importance: 0.012981008214945492
Name: connectivity, Importance: 0.012435273853077649
Name: station_count, Importance: 0.011413497914044614
Name: Artificial surfaces, Importance: 0.008652945702732338
Name: node_count, Importance: 0.007805786981597861


##### II. Random Forest regression

In [85]:
rf_model = RandomForestRegressor(n_estimators=50, random_state=0)
rf_model.fit(X_train_selected, y_train)
y_pred_rf = rf_model.predict(X_test_selected)

print(f'Random Forest MSE: {mean_squared_error(y_test, y_pred_rf)}')
print(f'Random Forest R²: {r2_score(y_test, y_pred_rf)}')

Random Forest MSE: 50968.24716690126
Random Forest R²: -0.2441482681637701


In [86]:
# XGBoost regression
xgb_model = XGBRegressor(n_estimators=100, random_state=0)
xgb_model.fit(X_train_selected, y_train)
y_pred_xgb = xgb_model.predict(X_test_selected)

print(f'XGBoost MSE: {mean_squared_error(y_test, y_pred_xgb)}')
print(f'XGBoost R²: {r2_score(y_test, y_pred_xgb)}')

XGBoost MSE: 39134.14200243303
XGBoost R²: 0.04472533970090109
